In [ ]:
from miditok import REMI
from miditok.utils import get_midi_programs
from miditoolkit import MidiFile
from pathlib import Path

# Creates the tokenizer and loads a MIDI
tokenizer = REMI()  # using the default parameters, read the documentation to customize your tokenizer
midi = MidiFile('Dataset/chopin/chp_op18.mid')

# Converts MIDI to tokens, and back to a MIDI
tokens = tokenizer(midi)  # automatically detects MIDIs and tokens before converting
converted_back_midi = tokenizer(tokens, get_midi_programs(midi))  # PyTorch / Tensorflow / Numpy tensors supported
# get path to processed data
token_BPE_path = Path('Dataset_tokenized_BPE')


In [ ]:
conv_back_midi = tokenizer("MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.json", get_midi_programs(midi))  

In [25]:
# Converts MIDI files to tokens saved as JSON files
midi_paths = list(Path('Dataset').glob('**/*.mid'))

tokenizer.tokenize_midi_dataset(midi_paths, Path('Dataset_tokenized'))

# Constructs the vocabulary with BPE, from the tokenized files
tokenizer.learn_bpe(
    vocab_size=500,
    tokens_paths=list(Path('Dataset_tokenized').glob("**/*.json")),
    start_from_empty_voc=False,
)

# Converts the tokenized musics into tokens with BPE
tokenizer.apply_bpe_to_dataset(Path('Dataset_tokenized'), Path('Dataset_tokenized_BPE'))

<ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
from miditok import REMI
from pathlib import Path

# Creates the tokenizer and list the file paths
tokenizer = REMI(special_tokens=["BOS", "EOS"])
midi_paths = list(Path('Final_Project', 'big_dataset').glob('**/*.mid'))

# A validation method to discard MIDIs we do not want
def midi_valid(midi) -> bool:
    if any(ts.numerator != 4 for ts in midi.time_signature_changes):
        return False  # time signature different from 4/*, 4 beats per bar
    if midi.max_tick < 10 * midi.ticks_per_beat:
        return False  # this MIDI is too short
    return True

# Converts MIDI files to tokens saved as JSON files
tokenizer.tokenize_midi_dataset(        # 2 velocity and 1 duration values
    midi_paths,
    Path('Final_Project', 'tokens_noBPE'),
    midi_valid,
)

# Learns the vocabulary with BPE
tokenizer.learn_bpe(
    vocab_size=512,
    tokens_paths=list(Path('Final_Project', 'tokens_noBPE').glob("**/*.json")),
    out_dir=Path('Final_Project', 'tokens_BPE'),
)

# Converts the tokenized musics into tokens with BPE
tokenizer.apply_bpe_to_dataset(Path('Final_Project', 'tokens_noBPE'), Path('Final_Project', 'tokens_BPE'))